<a href="https://colab.research.google.com/github/siyil/My-code/blob/master/Bootstrapttest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
#from scipy.interpolate import spline
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols, quantreg
from ds_tools import db
from ds_tools import df_to_snowflake
import datetime as dt
from statsmodels.distributions.empirical_distribution import ECDF
from patsy import dmatrices

In [0]:

getmetric = '''
select * from dm.firmware_upgrade_analysis_Lishui_daily_off_minutes_stat
'''
runregression1 = """
select * from dm.firmware_upgrade_analysis_Lishui_daily_off_minutes
"""
runregression2 = """
select * from dm.firmware_upgrade_analysis_Lishui_daily_off_minutes_AA
"""

samples = db.query_snowflake(getmetric,warehouse = 'COMPUTE_WH')
mydata1 = db.query_snowflake(runregression1,warehouse = 'COMPUTE_WH')
mydata2 = db.query_snowflake(runregression2,warehouse = 'COMPUTE_WH')

samples = samples.assign(pval = lambda x: 1-stats.t.cdf(np.abs(x.tstat), x.control_n + x.test_n - 2))
samples = samples.assign(pvala = lambda x: 1-stats.t.cdf(np.abs(x.tstata), x.controla_n + x.testa_n - 2))
#samples1 = samples.assign(alpha = 1)
#samples5 = samples.assign(alpha = 5)
#samples10 = samples.assign(alpha = 10)
#samples = samples1.append(samples5).append(samples10)

#samples = samples.assign(upper = lambda x: x.diff_mean.astype(float) + stats.t.ppf(1- x.alpha/200, x.control_n + x.test_n - 2)*x.sp*np.sqrt(1/x.control_n + 1/x.test_n))
#samples = samples.assign(lower = lambda x: x.diff_mean.astype(float) - stats.t.ppf(1- x.alpha/200, x.control_n + x.test_n - 2)*x.sp*np.sqrt(1/x.control_n + 1/x.test_n))


In [0]:
def sample(test, control):
    s1 = len(test)
    df = test.append(control)
    bootsample = random.choices(df.values.tolist(), k=len(df))
    t_boot = bootsample[0:s1]
    c_boot = bootsample[s1:]
    return t_boot, c_boot

def tstat(treatment, control, method = 'two-sample'):
    t_n = len(treatment)
    c_n = len(control)
    
    t_mean = np.mean(treatment)
    c_mean = np.mean(control)
    
    t_sd = np.std(treatment)
    c_sd = np.std(control)
    
    df = t_n + c_n - 2
    sp = np.sqrt(((t_n - 1)*(t_sd)**2 + (c_n - 1)*(c_sd)**2) / df) 
    
    tvalue = (t_mean-c_mean)/sp/np.sqrt(1/t_n+1/c_n)
    return tvalue

def diff(treatment, control):
    return np.mean(treatment) - np.mean(control)
  

def bootstrap_t_test(treatment, control, nboot = 2000):
    # tvalue = tstat(treatment, control)
    tvalue = diff(treatment, control)
    tboot = np.zeros(nboot)
    for i in range(nboot):
        random.seed(1000+i)
        t_boot, c_boot = sample(treatment, control)
        # tboot[i] = tstat(t_boot, c_boot)
        tboot[i] = diff(t_boot, c_boot)
    pvalue = np.sum(np.abs(tboot) >= np.abs(tvalue))/nboot
    return pvalue

In [55]:
experiments = list(mydata1['experiment_name'].unique())
n_result = len(experiments)+1
bootresults = {'experiment_name' : experiments + ['all'], 'pvalboot':[0]*n_result, 'pvalbootAA':[0]*n_result}
boot = pd.DataFrame(bootresults)

for exp in experiments:
  mytest1 = mydata1.loc[mydata1['experiment_name'] == exp].loc[mydata1['group_name'] == 'test', 'daily_off_minutes'].astype(float)
  mycontrol1 = mydata1.loc[mydata1['experiment_name'] == exp].loc[mydata1['group_name'] == 'control', 'daily_off_minutes'].astype(float)
  boot.loc[boot['experiment_name'] == exp, 'pvalboot'] = bootstrap_t_test(mytest1, mycontrol1)
  
  mytest2 = mydata2.loc[mydata2['experiment_name'] == exp].loc[mydata2['group_name'] == 'test', 'daily_off_minutes'].astype(float)
  mycontrol2 = mydata2.loc[mydata2['experiment_name'] == exp].loc[mydata2['group_name'] == 'control', 'daily_off_minutes'].astype(float)
  boot.loc[boot['experiment_name'] == exp, 'pvalbootAA'] = bootstrap_t_test(mytest2, mycontrol2)
  
  
mytest1 = mydata1.loc[mydata1['group_name'] == 'test', 'daily_off_minutes'].astype(float)
mycontrol1 = mydata1.loc[mydata1['group_name'] == 'control', 'daily_off_minutes'].astype(float)
boot.loc[boot['experiment_name'] == 'all', 'pvalboot'] = bootstrap_t_test(mytest1, mycontrol1)


mytest2 = mydata2.loc[mydata2['group_name'] == 'test', 'daily_off_minutes'].astype(float)
mycontrol2 = mydata2.loc[mydata2['group_name'] == 'control', 'daily_off_minutes'].astype(float)
boot.loc[boot['experiment_name'] == 'all', 'pvalbootAA'] = bootstrap_t_test(mytest2, mycontrol2)


samples = samples.merge(boot, on = 'experiment_name')
print(samples)

                                     experiment_name         test_mean  \
0  Firmware_Experiment_Berlin_Lishui_S_3.0GE_Z_E_...  196.326061647295   
1  Firmware_Experiment_Edmonton_Lishui_S_V3.0_Z_U...  233.974117711429   
2  Firmware_Experiment_Calgary_Lishui_S_V3.0_Z_U_...  177.901103664845   
3      Munich_Lishui_S_3.0GE_Z_E_0.2_0.1_0.1_0.1_0.2   45.330428739000   
4  Firmware_Experiment_Cologne_Lishui_S_3.0GE_Z_E...  353.756380956667   
5  Firmware_Experiment_Paris_Lishui_S_V3.0_Z_E_0....  392.689027501433   
6  Firmware_Experiment_Zurich_Lishui_S_V3.0_Z_E_0...  175.727917907937   
7                                                all  178.864786315772   
8   Frankfurt_Lishui_S_3.0GE_Z_E_0.2_0.1_0.1_0.1_0.2   36.461428706000   

       control_mean     test_sd  control_sd          diff_mean          sp  \
0  201.005527486991  271.326001  262.483855    -4.679465839696  266.938588   
1  257.474970518625  179.643918  204.353004   -23.500852807196  192.377850   
2  181.259251369091   94.

In [0]:
samples.to_csv('/Users/siyi.luo/Code/IoT Experiment/stats2.csv')